In [4]:
import numpy as np
from cmath import sqrt

In [5]:
zero = np.matrix('1;0')
one = np.matrix('0;1')

In [6]:
def kron(*matrices):
    assert len(matrices) > 0, "Cannot perform Kronecker product on no matrices"
    return reduce(np.kron, matrices[1:], matrices[0])

In [19]:
class Q(object):

    def __init__(self, state):
        state_as_row = state.flatten().tolist()[0]
        sq_mag = sum(a**2 for a in state_as_row)
        assert abs(1-sq_mag) < 0.00001, "Squared magnitudes must sum to 1"
        self.state = state

    def __add__(self, other):
        new_state = self.state + other.state
        as_row = new_state.flatten().tolist()[0]
        norm_factor = sum(p**2 for p in as_row)
        norm_new_state = 1/norm_factor * new_state
        return Q(norm_new_state)
    
    def apply_gate(self, gate):
        new_state = gate * self.state
        as_row = new_state.flatten().tolist()[0]
        norm_factor = sum(p**2 for p in as_row)
        norm_new_state = 1/norm_factor * new_state
        return Q(norm_new_state)
    
    def __repr__(self):
        return str(self.state)

In [35]:
I = np.eye(2)
X = np.matrix('0 1; 1 0')
Z = np.matrix('1 0; 0 -1')
H = 1/sqrt(2) * (X + Z)

cnot_10 = np.matrix('1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0')
cnot_01 = np.matrix('1 0 0 0; 0 0 0 1; 0 0 1 0; 0 1 0 0')

def get_entangled_pair():
    state = 1/sqrt(2) * (kron(zero, zero) + kron(one, one))
    return Q(state)

def gate(qubits, unitary, qidx):
    gate_seq = (unitary if idx == qidx else I for idx in range(qubits))
    return kron(*gate_seq)

In [41]:
# 1. start with an entangled pair. Alice owns qubit 0 and Bob owns qubit 1
state = get_entangled_pair()
print "1. start with an entangled pair."
print state

# 2. Alice wants to send ab = 10
a, b = 1, 0
# If a = 1, Alice applies Z to her qubit.
if a == 1:
    state = state.apply_gate(gate(2, Z, 0))
# If b = 1, Alice applies X to her qubit.
if b == 1:
    state = state.apply_gate(gate(2, X, 0))
print "\n2. Alice wants to send ab = {:}{:}".format(a,b)
print state

# 3. Alice sends her qubit to Bob.
print "\n3. Alice sends her qubit to Bob."

# 4. Bob applies CNOT with Alice's qubit as control.
# 2 qubit gate, qubit 0 is control, qubit 1 is target
state = state.apply_gate(cnot_10)
print "\n4. Bob applies CNOT with Alice's qubit as control."
print state

# 5. Bob applies H to Alice's qubit.
alice_h = gate(2, H, 0)
state = state.apply_gate(alice_h)
print "\n5. Bob applies H to Alice's qubit."
print state

# 6. Bob measures the qubits to get message ab.
print "\n6. Bob measures the qubits to get message ab."

1. start with an entangled pair.
[[ 0.70710678+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.70710678+0.j]]

2. Alice wants to send ab = 10
[[ 0.70710678+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [-0.70710678+0.j]]

3. Alice sends her qubit to Bob.

4. Bob applies CNOT with Alice's qubit as control.
[[ 0.70710678+0.j]
 [ 0.00000000+0.j]
 [-0.70710678+0.j]
 [ 0.00000000+0.j]]

5. Bob applies H to Alice's qubit.
[[ 0.+0.j]
 [ 0.+0.j]
 [ 1.+0.j]
 [ 0.+0.j]]

6. Bob measures the qubits to get message ab.
